#### 知识图谱分析

In [4]:
from collections import defaultdict
from data.data_aug import data_ag
from utils.load import load_train_data
import matplotlib
from matplotlib import pyplot as plt
import numpy as np

In [5]:
# 加载数据
triple_list, entity2id, relation2id = load_train_data('dataset/processed/')
# 设置字体为支持中文
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 中文黑体
matplotlib.rcParams['axes.unicode_minus'] = False  # 正常显示负号

In [6]:
# 加载三元组并进行分析
def load_triples(file_path):
    triples = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            h, r, t = line.strip().split()  # 三元组
            triples.append((h, r, t))  # 直接返回三元组中的字符串
    return triples

# 调用
train_triples = load_triples(f"dataset/processed/train.txt")
test_triples = load_triples(f"dataset/processed/test.txt")





1. 基本统计：分析实体和关系的数量

In [7]:
def basic_stats(entity2id, relation2id):
    num_entities = len(entity2id)
    num_relations = len(relation2id)

    print(f"实体总数: {num_entities}")
    print(f"关系总数: {num_relations}")

2. 关系频率分析：分析训练集中的关系频率，识别最(不)常见的关系。

In [8]:
def analyze_relation_frequencies(triples):
    relation_count = defaultdict(int)

    for h, r, t in triples:
        relation_count[r] += 1

    # 按照关系出现次数从高到低排序
    sorted_relations_by_count = sorted(relation_count.items(), key=lambda x: x[1], reverse=True)
    
    # 输出前5个最常见的关系
    print("前5个最常见的关系:")
    for relation, count in sorted_relations_by_count[:5]:
        print(f"关系: {relation}  出现次数: {count}")

    # 输出最不常见的5个关系
    print("最不常见的5个关系:")
    for relation, count in sorted_relations_by_count[-5:]:
        print(f"关系: {relation}  出现次数: {count}")
    
    # 获取所有关系的出现次数
    counts = [count for relation, count in sorted_relations_by_count]

    # 定义区间
    max_count = sorted_relations_by_count[0][1]
    bin_size = 10000  # 每个区间的大小
    bins = list(range(0, max_count + bin_size, bin_size))  # 以1000为区间进行划分
    relation_distribution = defaultdict(int)

    # 统计每个区间内有多少关系
    for count in counts:
        for i in range(len(bins) - 1):
            if bins[i] <= count < bins[i + 1]:
                relation_distribution[(bins[i], bins[i + 1])] += 1
                break

    # 输出各个区间内的关系数量
    print("关系出现次数的区间及其关系数量:")
    for (start, end), count in relation_distribution.items():
        print(f"{start} - {end}: {count} 个关系")

3. 实体连接分析：统计每个实体的度，找出在知识图谱中最活跃的实体。

In [9]:
def analyze_entity_degrees(triples):
    entity_degree = defaultdict(int)

    for h, r, t in triples:
        entity_degree[h] += 1
        entity_degree[t] += 1
    
    # 按照实体度从高到低排序
    sorted_entities_by_degree = sorted(entity_degree.items(), key=lambda x: x[1], reverse=True)
    
    # 输出度最大的5个实体
    print("度数最大的5个实体:")
    for entity, degree in sorted_entities_by_degree[:5]:
        print(f"实体: {entity}  度数: {degree}")
        
    # 输出度最小的5个实体
    print("度最小的5个实体:")
    for entity, degree in sorted_entities_by_degree[-5:]:
        print(f"实体: {entity}  度数: {degree}")
    
    # 获取度数数据
    degrees = list(entity_degree.values())
    # 定义区间
    bins = np.concatenate((np.arange(0, 501, 100), np.arange(500, 5001, 500)))  # 0-500细分为100, 其他区间500到5000
    # 计算每个区间的实体数量
    hist, edges = np.histogram(degrees, bins=bins)
    # 打印统计结果
    print("实体度数的区间及其实体数量:")
    for i in range(len(hist)):
        print(f'{edges[i]} - {edges[i+1]}: {hist[i]} 个实体')

4. 训练集和测试集划分分析

In [10]:
def analyze_train_test_split(train_triples, test_triples):
    train_entities = set([h for h, r, t in train_triples]).union(set([t for h, r, t in train_triples]))
    test_entities = set([h for h, r, t in test_triples]).union(set([t for h, r, t in test_triples]))
    train_relations = set([r for h, r, t in train_triples])
    test_relations = set([r for h, r, t in test_triples])

    print(f"训练集实体数: {len(train_entities)}")
    print(f"测试集实体数: {len(test_entities)}")
    print(f"训练集关系数: {len(train_relations)}")
    print(f"测试集关系数: {len(test_relations)}")


调用

#### 数据增强

In [11]:
# 实体和关系的数量
basic_stats(entity2id, relation2id)
print()
# 训练、测试集分析
analyze_train_test_split(train_triples, test_triples)
print()
# 关系频率分析
analyze_relation_frequencies(train_triples)
print()
# 实体度分析
analyze_entity_degrees(train_triples)

实体总数: 162336
关系总数: 47

训练集实体数: 145227
测试集实体数: 59857
训练集关系数: 47
测试集关系数: 47

前5个最常见的关系:
关系: Causes  出现次数: 57801
关系: MotivatedByGoal  出现次数: 44408
关系: HasSubevent  出现次数: 32722
关系: IsA  出现次数: 28883
关系: RelatedTo  出现次数: 24962
最不常见的5个关系:
关系: DefinedAs  出现次数: 73
关系: LocatedNear  出现次数: 64
关系: dbpedia/leader  出现次数: 29
关系: NotCapableOf  出现次数: 10
关系: EtymologicallyDerivedFrom  出现次数: 10
关系出现次数的区间及其关系数量:
50000 - 60000: 1 个关系
40000 - 50000: 1 个关系
30000 - 40000: 1 个关系
20000 - 30000: 3 个关系
10000 - 20000: 10 个关系
0 - 10000: 31 个关系

度数最大的5个实体:
实体: 睡觉  度数: 4462
实体: 吃饭  度数: 3549
实体: 人  度数: 2728
实体: 快乐  度数: 2261
实体: 开心  度数: 2189
度最小的5个实体:
实体: 抓虾子  度数: 1
实体: 什拉赫塔  度数: 1
实体: 踹你  度数: 1
实体: 盥洗完毕  度数: 1
实体: 黑胡椒猪排  度数: 1
实体度数的区间及其实体数量:
0 - 100: 144300 个实体
100 - 200: 517 个实体
200 - 300: 176 个实体
300 - 400: 77 个实体
400 - 500: 55 个实体
500 - 500: 0 个实体
500 - 1000: 73 个实体
1000 - 1500: 18 个实体
1500 - 2000: 5 个实体
2000 - 2500: 3 个实体
2500 - 3000: 1 个实体
3000 - 3500: 0 个实体
3500 - 4000: 1 个实体
4000 - 4500: 1 个实体
4500 - 5000: 0 个实体
